# 실무예제 2-2

## 다음은 2016년 항만별 선박입출황 현황이다. 선박수 속성은 구간화(Binning) 기법으로 평활화시키고, 선박톤수 속성은 선박수 속성과의 회귀(regression) 분석을 통하여 평활화시키시오.

### 데이터 파일 : ch2-2(선박입출항).csv

### 원본 투플수 : 30개

In [1]:
# ch2-2.py
import pandas as pd
import numpy as np  # 수학 및 과학 연산을 위한 numpy 패키지 임포트
import cx_Oracle      # Oracle DB 연동을 위한 cx_Oracle 패키지 임포트

# 데이터로드 (ch2-2.csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData = pd.read_csv('.jupyter/ch2-2(선박입출항).csv', encoding='CP949', engine='python')

In [2]:
rawData

,항만,입항선박수,입항선박톤수,출항선박수,출항선박톤수
0,부산,7301,105138280,7409,103857903
1,인천,2715,30716710,2716,30779186
2,평택.당진,1558,23153226,1536,22778109
3,경인항,28,126236,27,128344
4,동해.묵호,552,4202603,546,4039929
5,삼척,186,589721,189,599042
6,속초,15,6631,17,7887
7,옥계,164,572269,167,584674
8,호산,14,976358,13,890153
9,대산,1333,10800292,1324,10944114


In [3]:
# Oracle DB 연결
# 접속정보(connection string) : ID/PASS@CONNECTION_ALIAS
# CONNECTION_ALIAS : Oracle TNSNAMES.ORA 파일에 있는 접속정보 별칭(ALIAS)
conn_ora = cx_Oracle.connect("preswot/elqlfoq1234@AWS_ORA")

# DB 커서(Cursor) 선언
cur = conn_ora.cursor()

# 사용할 Oracle 소스 테이블명 지정
src_table = "d_base2_2"

# 데이터프레임(rawData)에 저장된 데이터를 Oracle 테이블(d_base2_2)에 입력하기 위한 로직
# d_base2_2 테이블 존재하는지 체크하는 함수
def table_exists(name=None, con=None):
    sql = "select table_name from user_tables where table_name='MYTABLE'".replace('MYTABLE', name.upper())
    df = pd.read_sql(sql, con)

    # 테이블이 존재하면 True, 그렇지 않으면 False 반환
    exists = True if len(df) > 0 else False
    return exists

# 테이블(d_base2_2) 생성 (테이블이 이미 존재한다면 TRUNCATE TABLE)
if table_exists(src_table, conn_ora):
    cur.execute("TRUNCATE TABLE " + src_table)
else:
    cur.execute("create table " + src_table + " ( \
               항만 varchar2(20) primary key, \
               입항선박수 number(5), \
               입항선박톤수 number(10), \
               출항선박수 number(5), \
               출항선박톤수 number(10))")

### cx_Oracle 패키지를 이용하여 Oracle DBMS와 연동
### raw 데이터를 Oracle 테이블에 저장하고, 이를 대상으로 SQL을 활용하여 평활화 작업을 수행
### Oracle SQL은 분석함수 등 데이터 평활화 처리에 유리한 기능들을 다수 포함되어 있어 평활화 구현이 용이
### read_sql() : SQL의 수행결과를 데이터프레임에 넣은 pandas 함수
### Oracle sqlplus를 이용하여 테이블 생성 확인

In [4]:
# Sequence 구조를 Dictionary 구조((element, value))로 변환하는 함수
# 예: ["Matt", 1] -> {'1':'Matt', '2':1}
# INSERT INTO ... VALUES (:1, :2, ...) 에서 바인드 변수값을 주기위해 Dictionary item 구조 사용
def convertSequenceToDict(list):
    dict = {}
    argList = range(1, len(list) + 1)
    for k, v in zip(argList, list):
        dict[str(k)] = v
    return dict

### convertSequenceToDict() : sequence 구조의 리스트를 입력받아 (element, value) 구조의 Dictionary를 반환
### 뒤의 data 변수 확인

In [5]:
# 데이터프레임에 저장된 데이터를 Oracle 테이블로 입력(insert)
cols = [k for k in rawData.dtypes.index]
colnames = ','.join(cols)
colpos = ', '.join([':' + str(i + 1) for i, f in enumerate(cols)])
insert_sql = 'INSERT INTO %s (%s) VALUES (%s)' % (src_table, colnames, colpos)

### SQL INSERT 문 생성

In [6]:
colnames

'항만,입항선박수,입항선박톤수,출항선박수,출항선박톤수'

In [7]:
colpos

':1, :2, :3, :4, :5'

In [8]:
insert_sql

'INSERT INTO d_base2_2 (항만,입항선박수,입항선박톤수,출항선박수,출항선박톤수) VALUES (:1, :2, :3, :4, :5)'

In [9]:
# INSERT INTO ... VALUES (:1, :2, ...)의 바인드 변수 값을 저장하는 Dictionary 구조 생성
data = [convertSequenceToDict(rec) for rec in rawData.values]

In [10]:
data

[{'1': '부산', '2': 7301, '3': 105138280, '4': 7409, '5': 103857903},
 {'1': '인천', '2': 2715, '3': 30716710, '4': 2716, '5': 30779186},
 {'1': '평택.당진', '2': 1558, '3': 23153226, '4': 1536, '5': 22778109},
 {'1': '경인항', '2': 28, '3': 126236, '4': 27, '5': 128344},
 {'1': '동해.묵호', '2': 552, '3': 4202603, '4': 546, '5': 4039929},
 {'1': '삼척', '2': 186, '3': 589721, '4': 189, '5': 599042},
 {'1': '속초', '2': 15, '3': 6631, '4': 17, '5': 7887},
 {'1': '옥계', '2': 164, '3': 572269, '4': 167, '5': 584674},
 {'1': '호산', '2': 14, '3': 976358, '4': 13, '5': 890153},
 {'1': '대산', '2': 1333, '3': 10800292, '4': 1324, '5': 10944114},
 {'1': '보령', '2': 65, '3': 1324916, '4': 62, '5': 1286459},
 {'1': '태안', '2': 52, '3': 1178735, '4': 55, '5': 1254622},
 {'1': '군산', '2': 528, '3': 6688062, '4': 525, '5': 7176882},
 {'1': '장항', '2': 81, '3': 73959, '4': 81, '5': 74734},
 {'1': '목포', '2': 1040, '3': 5028247, '4': 1074, '5': 6095534},
 {'1': '완도', '2': 304, '3': 161468, '4': 305, '5': 161830},
 {'1': '여수', 

In [11]:
# 바인드 변수와 Dictionary 데이터구조를 활용하여 Bulk Insertion 구현
cur.executemany(insert_sql, data)

# csv 파일 데이터의 Oracle 테이블 입력 완료 
conn_ora.commit()

### executemany() : 복수 개의 투플을 한 번에 Oracle 테이블에 입력하는 함수
### commit() : Oracle 테이블에 입력을 승인하는 함수

## 1) 구간화(Binning) - 평균값 평활화 (Smoothing by Bin Means)

In [12]:
# 평균값 평활화 (동일 너비 방식, 구간너비 : 500) 
result_df = pd.read_sql("select 항만 \
               , 입항선박수 \
               , round(avg(입항선박수) over (partition by floor(입항선박수/500)), 0) 입항선박수_평활 \
               , 출항선박수 \
               , round(avg(출항선박수) over (partition by floor(출항선박수/500)), 0) 출항선박수_평활 \
           from " + src_table, con=conn_ora)

### avg() over (partition by ...) : partition by에 의해 파티션된 그룹별 평균을 구하는 Oracle 분석함수
### floor(입항선박수/500) : 입항선박수/500의 결과값보다 크지 않은 정수 중에서 가장 최대값을 산출하는 Oracle 함수
### round(A,0) : A값을 소수첫째자리에서 반올림하는 Oracle 함수

In [13]:
# 결과보기(첫 10개행만 출력)
result_df.head(10)

,항만,입항선박수,입항선박수_평활,출항선박수,출항선박수_평활
0,장승포,3,129,3,130
1,통영,178,129,180,130
2,태안,52,129,55,130
3,보령,65,129,62,130
4,호산,14,129,13,130
5,옥계,164,129,167,130
6,속초,15,129,17,130
7,삼척,186,129,189,130
8,경인항,28,129,27,130
9,옥포,422,129,426,130


## 2) 구간화(Binning) - 중앙값 평활화 (Smoothing by Bin Medians)

In [14]:
# 중앙값 평활화 (동일 너비 방식, 구간너비 : 500)
# 입항선박수 중앙값을 구하는 뷰(view)
cur.execute("create or replace view 입항_median_view \
           as \
           select 항만 \
                , 입항선박수 \
                , bin_id /* 구간 id */ \
                , rnum /* 구간 내의 위치 */ \
                , floor((cnt + 1) / 2) med1 /* 구간의 중앙 위치(왼쪽) */ \
                , ceil((cnt + 1) / 2) med2 /* 구간의 중앙 위치(오른쪽) */ \
           from ( select 항만 \
                       , 입항선박수 \
                       , floor(입항선박수 / 500) bin_id \
                       , row_number() over (partition by floor(입항선박수 / 500) order by 입항선박수) rnum \
                       , count(*) over (partition by floor(입항선박수 / 500)) cnt \
                  from " + src_table + " )")

# 출항선박수 중앙값을 구하는 뷰(view)
cur.execute("create or replace view 출항_median_view \
           as \
           select 항만 \
                , 출항선박수 \
                , bin_id /* 구간 id */ \
                , rnum /* 구간 내의 위치 */ \
                , floor((cnt + 1) / 2) med1 /* 구간의 중앙 위치(왼쪽) */ \
                , ceil((cnt + 1) / 2) med2 /* 구간의 중앙 위치(오른쪽) */ \
           from ( select 항만 \
                       , 출항선박수 \
                       , floor(출항선박수 / 500) bin_id \
                       , row_number() over(partition by floor(출항선박수 / 500) order by 출항선박수) rnum \
                       , count(*) over(partition by floor(출항선박수 / 500)) cnt \
                  from " + src_table + " )")

### 중앙값 평활화를 위해서 구간 크기를 정하고, 각 구간의 중앙값을 구함 
### 각 구간의 중앙값은 해당 구간에 속한 속성값들을 정렬했을 때, 가운데 위치한 값을 의미
### 중앙값을 구하기 위해서 우선 각 구간의 중앙 위치 정보를 구함
### 중앙 위치 정보를 구하기 위해서 Oracle 분석함수 row_number() over ()와 count() over ()를 활용
#### row_number() over () : 순번
#### count() over () : 구간크기
### 속성값이 홀수개일 때는 중앙위치가 하나이고, 짝수개일 때는 2개임. 이를 floor()와 ceil()을 이용하여 구함
#### floor() : 소수점을 버리는 함수이고
#### ceil() : 소수점을 올려 정수를 만드는 함수
### 속성값이 홀수개일 때는 med1과 med2가 동일한 값을 가짐

In [15]:
# 입항선박수와 출항선박수 평활(중앙)값 결합
result_df = pd.read_sql("select 항만 \
                           , min(decode(gubun, 1, col1)) 입항선박수 \
                           , min(decode(gubun, 1, col2)) 입항선박수_평활 \
                           , min(decode(gubun, 2, col1)) 출항선박수 \
                           , min(decode(gubun, 2, col2)) 출항선박수_평활 \
                      from ( select 1 gubun /* 입항선박수 중앙값 구하기 */ \
                                  , a.항만 \
                                  , a.입항선박수 col1 \
                                   /* b.입항선박수 : 구간 내 중앙(왼쪽)값, c.입항선박수 : 구간 내 중앙(오른쪽)값 */ \
                                  , round((b.입항선박수+c.입항선박수) / 2, 0) col2 \
                             from 입항_median_view a, 입항_median_view b, 입항_median_view c \
                             where a.bin_id = b.bin_id and a.med1 = b.rnum \
                                 and a.bin_id = c.bin_id and a.med2 = c.rnum \
                             union all \
                             select 2 gubun /* 출항선박수 중앙값 구하기 */ \
                                  , a.항만 \
                                  , a.출항선박수 col1 \
                                   /* b.출항선박수 : 구간 내 중앙(왼쪽)값, c.출항선박수 : 구간 내 중앙(오른쪽)값 */ \
                                  , round((b.출항선박수 + c.출항선박수) / 2, 0) col2 \
                             from 출항_median_view a, 출항_median_view b, 출항_median_view c \
                             where a.bin_id = b.bin_id and a.med1 = b.rnum \
                                 and a.bin_id = c.bin_id and a.med2 = c.rnum ) \
                      group by 항만", con=conn_ora)

### 중앙위치에서의 속성값을 구함
### 속성값이 홀수개일 때는 가운데 값이고 짝수개일 때는 두 개의 가운데 값의 평균임
### 다만, 가운데 값이 하나라 할지라도 두 개로 가정하여 평균을 구함 (하나의 값을 동일 값 두 개로 봄)
### 두 개 위치의 속성값을 찾아야하므로 동일 뷰를 두번 조인함

In [16]:
# 결과보기(첫 10개행만 출력)
result_df.head(10)

,항만,입항선박수,입항선박수_평활,출항선박수,출항선박수_평활
0,속초,15,81,17,81
1,태안,52,81,55,81
2,포항,967,652,947,658
3,광양,4050,4098,4078,4121
4,장승포,3,81,3,81
5,삼천포,222,81,221,81
6,삼척,186,81,189,81
7,호산,14,81,13,81
8,고현,587,652,583,658
9,평택.당진,1558,1558,1536,1536


## 3) 구간화(Binning) - 경계값 평활화 (Smoothing by Bin Boundaries)

In [17]:
# 경계값 평활화 (동일 너비 방식, 구간너비 : 500)
result_df = pd.read_sql("select 항만 \
                          , 입항선박수 \
                          , case when (입항선박수 - 입항_경계_최소) < (입항_경계_최대 - 입항선박수) then 입항_경계_최소 \
                            else 입항_경계_최대 end 입항선박수_평활 \
                          , 출항선박수 \
                          , case when (출항선박수 - 출항_경계_최소) < (출항_경계_최대 - 출항선박수) then 출항_경계_최소 \
                            else 출항_경계_최대 end 출항선박수_평활 \
                      from ( \
                             select 항만 \
                                  , 입항선박수 \
                                  , floor(입항선박수/500)*500 입항_경계_최소 \
                                  , floor(입항선박수/500)*500 + 500 입항_경계_최대 \
                                  , 출항선박수 \
                                  , floor(출항선박수/500)*500 출항_경계_최소 \
                                  , floor(출항선박수/500)*500 + 500 출항_경계_최대 \
                             from " + src_table + " )", con=conn_ora)

### 우선, 최소경계값과 최대경계값을 구한 다음
### 이를 대상으로 실제값과 경계값들의 차이를 구하고 이를 비교하여 두 경계값 중 하나를 선택함
### case when A then B else C end : 조건식 A가 참이면 B값을 그렇지 않으면 C값을 선택하는 SQL 구문

In [18]:
# 결과보기(첫 10개행만 출력)
result_df.head(10)

,항만,입항선박수,입항선박수_평활,출항선박수,출항선박수_평활
0,부산,7301,7500,7409,7500
1,인천,2715,2500,2716,2500
2,평택.당진,1558,1500,1536,1500
3,경인항,28,0,27,0
4,동해.묵호,552,500,546,500
5,삼척,186,0,189,0
6,속초,15,0,17,0
7,옥계,164,0,167,0
8,호산,14,0,13,0
9,대산,1333,1500,1324,1500


## 4) 회귀분석에 의한 평활화

In [2]:
# 선형회귀분석을 위한 scipy 패키지 중 stats 모듈 임포트
from scipy import stats

In [3]:
# 데이터프레임 복사 : rawData -> result_df
# copy=True 이면 별도의 공간에 데이터프레임 생성
result_df = pd.DataFrame(rawData, copy=True)

In [4]:
# 회귀분석에 의한 평활화 (입출항선박수와 입출항선박톤수 간 회귀분석)
# 회귀분석 시, 결측치나 수의 표현, 범위 상이 문제로 인한 경고메시지 출력 방지
np.seterr(invalid='ignore')

# stats.linregress(x, y) : y = slope * x + intercept 형식의 선형함수를 찾아주는 stats 모듈 함수로 다섯 개의 값을 반환
slope, intercept, r_value, p_value, std_err = stats.linregress(result_df['입항선박수'], result_df['입항선박톤수'])

# 입항선박수에 의한 입항선박톤수 산출
result_df['입항선박톤수'] = result_df['입항선박수']*slope + intercept

slope, intercept, r_value, p_value, std_err = stats.linregress(rawData['출항선박수'], rawData['출항선박톤수'])

# 출항선박수에 의한 출항선박톤수 산출
result_df['출항선박톤수'] = result_df['출항선박수']*slope + intercept

### stats.linregress(x, y) : 속성 x에 대한 속성 y의 선형회귀분석을 수행하는 scipy 패키지 stats 모듈함수이다. 5개의 결과값을 반환하는데, 『y = slope * x + intercept』 형태의 선형함수식으로 해석됨.
### 『result_df.loc['입항선박톤수'] = result_df['입항선박수']*slope + intercept』 수행을 통하여 rawData 데이터프레임 각 행의 입항선박수 대비 입항선박톤수를 구함
### 위 코드 수행과정에서 『RuntimeWarning: invalid value encountered in greater』와 같은 Runtime Warning 메시지가 나타날 수 있다. 이는 numpy 패키지에서 주로 발생하는데, 결측치(NaNs) 또는 수의 표현이나 범위 상의 문제로 발생한다. Warning 메시지가 나타나지 않도록 하려면, 『np.seterr(invalid='ignore')』 코드를 추가시켜 주면 됨

In [5]:
# 결과보기(첫 10개행만 출력)
result_df.head(10)

,항만,입항선박수,입항선박톤수,출항선박수,출항선박톤수
0,부산,7301,9.526956e+07,7409,9.560901e+07
1,인천,2715,3.363945e+07,2716,3.337250e+07
2,평택.당진,1558,1.809082e+07,1536,1.772385e+07
3,경인항,28,-2.470471e+06,27,-2.287845e+06
4,동해.묵호,552,4.571434e+06,546,4.594906e+06
5,삼척,186,-3.471481e+05,189,-1.394716e+05
6,속초,15,-2.645174e+06,17,-2.420460e+06
7,옥계,164,-6.428006e+05,167,-4.312260e+05
8,호산,14,-2.658613e+06,13,-2.473507e+06
9,대산,1333,1.506710e+07,1324,1.491240e+07


In [32]:
# 커서(cursor) 종료
cur.close()

# Oracle connection 종료
conn_ora.close()

### Oracle DB 세션(session) 종료 : 『ch2-2.py』의 마지막 코드 부분으로 DB 커서를 종료하고 Oracle connection을 종료함